In [1]:
import pandas as pd
import numpy as np

In [2]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

In [3]:
from azureml.core import Dataset

In [4]:
dataset = Dataset.get_by_name(ws, name='credit_card_data')
data = dataset.to_pandas_dataframe()

In [5]:
from sklearn.model_selection import train_test_split

y_df = data.pop("Class")
x_df = data

x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

In [6]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 2,
    "experiment_timeout_minutes": 15,
    "enable_early_stopping": True,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

In [7]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='classification',
                             debug_log='automated_ml_errors.log',
                             X=x_train.values,
                             y=y_train.values.flatten(),
                             **automl_settings)

WARNING - The AutoMLConfig inputs you have specified will soon be deprecated. Please use the AutoMLConfig shown in our documentation: https://aka.ms/AutoMLConfig


In [8]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "credit-experiment")
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_42870315-8f07-4838-be3f-eb2de8c73ba8

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  Classes in the training data are imbalanced.
PARAMETERS:   Size of the smallest class : 385, number of samples in the training data : 227845
              
TYPE:         Missing values imputation
STATUS:       PASSED
DESCRI

In [9]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
#Retrieve the best model
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: credit-experiment,
Id: AutoML_42870315-8f07-4838-be3f-eb2de8c73ba8_4,
Type: None,
Status: Completed)
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, force_text_dnn=None,
        is_cross_validation=None, is_onnx_compatible=None, logger=None,
        obser...      silent=True, subsample=1, subsample_for_bin=200000,
          subsample_freq=0, verbose=-10))])
